# **IMPORT LIBS**

In [1]:
import keras
import pandas as pd
import numpy as np
import warnings
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
#from skimage.transform import rotate, shear, zoom
#from imgaug import augmenters as iaa
import tensorflow as tf
import math

import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

warnings.filterwarnings('ignore')

#**LOAD THE MODEL**

In [ ]:
#model_lstm = tf.keras.models.load_model('./lstm_model_v40.h5')

# (Optional) Print a summary of the model architecture
#model_lstm.summary()

# **AUX FUNCTIONS**

In [2]:
def save_manovers_positions_to_csv_file(gps_positions, manovers, filename):
  output = np.zeros_like(gps_positions)

  # Iterate through the elements of arr2
  for i in range(len(manovers)):
    # Check if the element in arr2 is 1
    if manovers[i] == 1:
      # Copy the corresponding values from arr1 to the output array
      output[i] = gps_positions[i]

  output = output[~np.all(output == 0, axis=1)]
  np.savetxt(filename, output, delimiter=',', fmt='%.9f')



def separate_positives_negatives(data):
  # Ensure the input is converted to a NumPy array for easier manipulation
  data = np.array(data)

  # Create two empty arrays to store positive and negative values
  positives = np.zeros_like(data)
  negatives = np.zeros_like(data)

  # Use boolean indexing to separate positive and negative values
  positives[data > 0] = data[data > 0]
  negatives[data < 0] = -data[data < 0]

  # Combine the positive and negative values into a single 2D array
  return (positives, negatives)

def normalize_between_0_and_max(data):
  max_value = np.max(data)
  return data / max_value

def normalize_between_0_and_max_v2(data, max_value):
  return data / max_value

def split_train_test(data, test_size=0.2):
  # Check if test_size is between 0 and 1
  if test_size < 0 or test_size > 1:
    raise ValueError("test_size must be between 0 and 1.")

  # Get the number of samples
  num_samples = data.shape[0]

  # Calculate the number of samples for each set
  train_size = int(num_samples * (1 - test_size))
  test_size = num_samples - train_size

  # Randomly shuffle the data for better splitting (optional)
  #np.random.shuffle(data)

  # Split the data into training and test sets
  train_data = data[:train_size]
  test_data = data[train_size:]

  return train_data, test_data

def y_classification(data, threshold):
  classification = np.zeros_like(data, dtype=int)  # Initialize output array

  for col in range(0, 12):  # Loop through each column
    max_value = np.max(data[:, col])
    threshold_pos = max_value * threshold
    classification[:, col] = np.where(data[:, col] >= threshold_pos, 1, 0)

  return classification

def max_of_vectors(vec1, vec2, vec3, vec4, vec5, vec6):
  # Combine all vectors into a single array
  all_vectors = np.array([vec1, vec2, vec3, vec4, vec5, vec6])

  # Find the maximum value in the array
  max_value = np.max(all_vectors)

  return max_value

def has_one(data):
  """
  This function receives a numpy array and returns a new array
  with 1 if the correspondent row of input array has at least one cellule with 1.
  In other case the cellule is 0.

  Args:
      data: A numpy array of shape (n, 12) with 0 or 1 values in each cell.

  Returns:
      A numpy array of shape (n, 1) with 1s where the corresponding row in data has at least one 1, and 0s otherwise.
  """
  # We sum each row, and any value greater than zero indicates at least one 1 in that row
  return np.sum(data, axis=1)[:, np.newaxis] > 0

# **IMPORT DATA**

In [45]:
#df = pd.read_csv('Dataset-viagens-longas.csv')
#df = pd.read_csv('Abrantes-Leiria.csv')
df = pd.read_csv('dataset-all.csv')

acelX = df['accelerometerXAxis']
acelY = df['accelerometerYAxis']
acelZ = df['accelerometerZAxis']

gyrX = df['gyroscopeXAxis']
gyrY = df['gyroscopeYAxis']
gyrZ = df['gyroscopeZAxis']

latitude = df['latitude']
longitude = df['longitude']

In [4]:
print(df['accelerometerXAxis'].describe())
print(df['accelerometerYAxis'].describe())
print(df['accelerometerZAxis'].describe())
print(df['gyroscopeXAxis'].describe())
print(df['gyroscopeYAxis'].describe())
print(df['gyroscopeZAxis'].describe())

count    150070.000000
mean          0.001342
std           1.264318
min         -11.536232
25%          -0.547894
50%           0.005779
75%           0.559436
max          14.109398
Name: accelerometerXAxis, dtype: float64
count    150070.000000
mean          0.055583
std           0.932284
min         -13.825549
25%          -0.397755
50%           0.042851
75%           0.495079
max          11.939904
Name: accelerometerYAxis, dtype: float64
count    150046.000000
mean          0.087325
std           1.411255
min         -20.017138
25%          -0.612250
50%           0.050575
75%           0.742957
max          16.155849
Name: accelerometerZAxis, dtype: float64
count    150070.000000
mean          0.000406
std           0.099782
min          -2.247918
25%          -0.040928
50%          -0.000305
75%           0.040317
max           2.192789
Name: gyroscopeXAxis, dtype: float64
count    150070.000000
mean          0.004894
std           0.172807
min          -2.713842
25%         

# **SEPARATE DATA BY MANOVER**

In [46]:
turnRightX, turnLeftX = separate_positives_negatives(acelX)

accelY, breakY = separate_positives_negatives(acelY)

positiveZ, negativeZ = separate_positives_negatives(acelZ)

gyrPositiveX, gyrNegativeX = separate_positives_negatives(gyrX)
gyrPositiveY, gyrNegativeY = separate_positives_negatives(gyrY)
gyrPositiveZ, gyrNegativeZ = separate_positives_negatives(gyrZ)

In [ ]:
turnRightX.shape

(150070,)

# **CLASSIFICATION BASED ON NOT NORMALIZED VALUES**

In [ ]:
#newArray1 = np.array(list(zip(turnRightX, turnLeftX, accelY, breakY, positiveZ, negativeZ, gyrPositiveX, gyrNegativeX, gyrPositiveY, gyrNegativeY, gyrPositiveZ, gyrNegativeZ)))

#y = y_classification(newArray1, 0.25)
#print (np.sum(y, axis=0))
#np.savetxt("Y.csv", y, delimiter=',', fmt='%.0i')

# **NORMALIZE DATA**

In [47]:
max_accel = max_of_vectors(turnRightX, turnLeftX, accelY, breakY, positiveZ, negativeZ)
max_gyr = max_of_vectors(gyrPositiveX, gyrNegativeX, gyrPositiveY, gyrNegativeY, gyrPositiveZ, gyrNegativeZ)

turnRightXn = normalize_between_0_and_max_v2(turnRightX, max_accel)
turnLeftXn = normalize_between_0_and_max_v2(turnLeftX, max_accel)
accelYn = normalize_between_0_and_max_v2(accelY, max_accel)
breakYn = normalize_between_0_and_max_v2(breakY, max_accel)
positiveZn = normalize_between_0_and_max_v2(positiveZ, max_accel)
negativeZn = normalize_between_0_and_max_v2(negativeZ, max_accel)
gyrPositiveXn = normalize_between_0_and_max_v2(gyrPositiveX, max_gyr)
gyrNegativeXn = normalize_between_0_and_max_v2(gyrNegativeX, max_gyr)
gyrPositiveYn = normalize_between_0_and_max_v2(gyrPositiveY, max_gyr)
gyrNegativeYn = normalize_between_0_and_max_v2(gyrNegativeY, max_gyr)
gyrPositiveZn = normalize_between_0_and_max_v2(gyrPositiveZ, max_gyr)
gyrNegativeZn = normalize_between_0_and_max_v2(gyrNegativeZ, max_gyr)

# **CREATE AN ARRAY WITH ALL DATA**

In [48]:
newArray = np.array(list(zip(turnRightXn, turnLeftXn, accelYn, breakYn, positiveZn, negativeZn, gyrPositiveXn, gyrNegativeXn, gyrPositiveYn, gyrNegativeYn, gyrPositiveZn, gyrNegativeZn)))

In [ ]:
newArray.shape

In [49]:
y = y_classification(newArray, 0.3)
print (np.sum(y, axis=0))
np.savetxt("Y.csv", y, delimiter=',', fmt='%.0i')

[ 789 2127  408  247  896  191   45   30   10  142   88    6]


# **SHOW MANOVERS ON GOOLGE MAP**

In [ ]:
positions = np.array(list(zip(latitude, longitude)))
manover_accelY = y[:, 2]
manover_breakY = y[:, 3]
manover_turnRightXn = y[:, 0]
manover_turnLeftXn = y[:, 1]
gyrPositiveZn = y[:, 10]
gyrNegativeZn = y[:, 11]
save_manovers_positions_to_csv_file(positions, manover_accelY, "accelY.csv")
save_manovers_positions_to_csv_file(positions, manover_breakY, "breakY.csv")
save_manovers_positions_to_csv_file(positions, manover_turnRightXn, "turnRightX.csv")
save_manovers_positions_to_csv_file(positions, manover_turnLeftXn, "turnLeftX.csv")
save_manovers_positions_to_csv_file(positions, gyrPositiveZn, "gyrPositZ.csv")
save_manovers_positions_to_csv_file(positions, gyrNegativeZn, "gyrNegZ.csv")

# **PLOT MANOVERS**

In [ ]:
plt.plot(y[:, 2], marker='.', linestyle='none')
plt.plot(y[:, 3], marker='.', linestyle='none')

plt.legend(['Accel Y', 'Break Y'])
plt.show()

# **SEPARATE DATA IN TRAIN AND TEST**

In [50]:
x_treino, x_teste = split_train_test(newArray, test_size=0.2)

y_treino, y_teste = split_train_test(y, test_size=0.2)

# **CREATE THE INPUT TENSORES DATA**

In [51]:
treino = x_treino.reshape(x_treino.shape[0], 1, x_treino.shape[1])
teste = x_teste.reshape(x_teste.shape[0], 1, x_teste.shape[1])

# **CREATE THE MODEL**

In [15]:
#K.clear_session()

#model_lstm = Sequential()
#model_lstm.add(LSTM(100, input_shape=(1, treino.shape[2]), activation='relu', return_sequences=True))
#model_lstm.add(LSTM(32, activation='relu', return_sequences=True))
#model_lstm.add(LSTM(100, activation='sigmoid'))
#model_lstm.add(Dense(y_treino.shape[1], activation='sigmoid'))

In [52]:
from tensorflow.keras.layers import LSTM, Dense, Dropout

model_lstm = Sequential()
model_lstm.add(LSTM(50, input_shape=(1, treino.shape[2]), return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(50))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(y_treino.shape[1], activation='sigmoid'))

# **COMPILE THE MODEL**

In [53]:
model_lstm.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
#model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)

# **TRAIN THE MODEL**

In [ ]:
#model_lstm_output = model_lstm.fit(treino, y_treino, epochs=10, batch_size=4, verbose=1, shuffle=False, callbacks=[early_stop], validation_data=(teste, y_teste))
#model_lstm_output = model_lstm.fit(treino, y_treino, epochs=3, batch_size=8, verbose=1, shuffle=False, callbacks=[early_stop])

In [54]:
# Define callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)

# Train the model
model_lstm_output = model_lstm.fit(treino, y_treino, epochs=9, batch_size=32, validation_split=0.2, shuffle=True, callbacks=[early_stopping, reduce_lr])

Epoch 1/9
3002/3002 [==============================] - 28s 8ms/step - loss: 0.0093 - accuracy: 0.2080 - val_loss: 0.0038 - val_accuracy: 0.0244 - lr: 0.0010
Epoch 2/9
3002/3002 [==============================] - 20s 7ms/step - loss: 0.0029 - accuracy: 0.0668 - val_loss: 0.0038 - val_accuracy: 0.0244 - lr: 0.0010
Epoch 3/9
3002/3002 [==============================] - 22s 7ms/step - loss: 0.0024 - accuracy: 0.0137 - val_loss: 0.0020 - val_accuracy: 0.0289 - lr: 0.0010
Epoch 4/9
3002/3002 [==============================] - 21s 7ms/step - loss: 0.0019 - accuracy: 0.0189 - val_loss: 0.0019 - val_accuracy: 0.0292 - lr: 0.0010
Epoch 5/9
3002/3002 [==============================] - 19s 6ms/step - loss: 0.0019 - accuracy: 0.0192 - val_loss: 0.0019 - val_accuracy: 0.0293 - lr: 0.0010
Epoch 6/9
3002/3002 [==============================] - 20s 7ms/step - loss: 0.0015 - accuracy: 0.2593 - val_loss: 0.0015 - val_accuracy: 0.9292 - lr: 0.0010
Epoch 7/9
3002/3002 [==============================] - 23s

# **SHOW THE RESULTS**

In [ ]:
plt.plot(model_lstm_output.history['loss'])
plt.plot(model_lstm_output.history['val_loss'])
plt.title('Historico de treino')
plt.xlabel('Epocas de treino')
plt.ylabel('Função custo')
plt.legend(['Erro treino', 'Erro teste'])
plt.show()

In [ ]:
#accuracy = model_lstm.evaluate(teste, y_teste)[1]  # Assuming accuracy is the second metric
#print('Test Accuracy:', accuracy)

# **TEST THE NETWORK**

In [44]:
loss, accurary = model_lstm.evaluate(teste, y_teste, batch_size=16)
print('Test loss/accurary:', loss, accurary)


1876/1876 [==============================] - 4s 2ms/step - loss: 3.8539e-04 - accuracy: 0.9841
Test loss/accurary: 0.0003853856469504535 0.9840741157531738


In [ ]:
teste[0]

In [ ]:
teste.shape

In [ ]:
i = 50000
for i in range(100):
    a = x_teste[i]
    b = a.reshape(1, 1, 12)

    # Make predictions on new data
    prediction = model_lstm.predict(b)
    #predicted_class = label_encoder.inverse_transform(prediction)[0]

    np.round(prediction, decimals=1, out=prediction)
    np.round(x_teste[i], decimals=1, out=x_teste[i])
    #print("Value:", newArray[i + start])
    if (np.sum(y_teste[i]) > 0):
      print("X [:", x_teste[i])
      print("Y [:", y_teste[i])
      print("PC:", prediction)
      print (i)
    i = 1 + 1
#PREDICTIONS WITH COLAB MODEL
#prediction = model.predict(teste)
#print("Predicted class:", prediction)
#print("Predicted class:", predicted_class)

In [ ]:
teste_value = np.array([0., 0.363, 0.313, 0., 0., 0.31, 0.393, 0., 0., 0.244, 0.247, 0.])
teste_value = teste_value.reshape(1, 1, 12)

# Make predictions on new data
prediction = model_lstm.predict(teste_value)
np.round(prediction, decimals=2, out=prediction)

print("Value    :", teste_value[0][0])
print("Predicted:", prediction[0])


1/1 [==============================] - 1s 822ms/step
Value    : [0.    0.363 0.313 0.    0.    0.31  0.393 0.    0.    0.244 0.247 0.   ]
Predicted: [0.04 1.   0.02 0.04 0.   0.01 0.01 0.01 0.01 0.01 0.44 0.01]


# **SAVE THE MODEL**

In [ ]:
import os
model_lstm.save(os.path.join(".", 'lstm_model_v23.h5'))

NameError: name 'model_lstm' is not defined